# Submission 2: Sentimen Analisis Timnas Sepak Bola Indonesia pada Masa Kepelatihan Shin-Tae Young
Nama: Fajar Ramadhan

Username dicoding: fajar_ramadhan_bbk

| | Deskripsi |
| ----------- | ----------- |
| Dataset | [Analisis Sentimen Timnas Sepakbola di Era STY](https://www.kaggle.com/datasets/pajarbebek/analisis-sentimen-timnas-sepakbola-di-era-sty) |
| Masalah | Sepak bola merupakan olahraga paling populer di Indonesia.Supporter sepak bola Indonesia merupakan salah satu supporter terbesar di dunia. Saat ini, performa timnas sepak bola Indonesia meningkat tajam sejak kehadiran pelatih kepala Shin-Tae Young. Hal ini tak lepas dari beberapa kebijakannya yang cukup berbeda dari pelatih lainnya, seperti naturalisasi hingga pemotongan 2 generasi. Hal ini menimbulkan perdebatan di kalangan masyarakat, ada yang mendukung, dan ada pula yang menyayangkannya. maka dari itu dibuatkan sebuah pemodelan untuk melakukan analisis sentimen terhadap komentar masyarakat pada media sosial X|
| Solusi machine learning | Membuat sebuah model yang dapat mengetahui sentimen masyakarakat lewat komentar. Hal ini dapat dimanfaatkan bagi *stakeholder* yang terlibat dalam timnas sepakbola untuk mengambil keputusan tentang perpanjangan kontrak kepala pelatih STY.
| Metode pengolahan | Data diolah pada komponen transform dan trainer dengan cara membersihkan komentar dari noise seperti perubahan huruf besar menjadi huruf kecil, serta penghapusan punctuation. Lalu label juga diubah dalam tipe tensor agar dapat diproses oleh pipeline. |
| Arsitektur model | model terdiri dari beberapa layer dengan tugas berbeda, seperti layer pertama terdapat layer TextVectorization untuk mengubah string menjadi bentuk numerik, lalu dilanjut dengan embedding, global average pooling, serta dense |
| Metrik evaluasi | metrik yang digunakan yaitu *Area Under Curve*, *False Positive* dan *Negative*, *True Positive* dan *Negative,* *Binary Accuracy*, serta *binary crossentropy*|
| Performa model | Model yang dilatih mendapatkan hasil yang baik dengan persentase binary accuracy 95% dengan total data validasi (*example count*) 76. Namun, jika ditinjau dari hasil evaluasi masih dibutuhkan peningkatan kualitas model. Pada metriks seperti AUC, *false negative*, dan *binary accuracy* dinilai masih kurang baik. Nilai AUC tidak lebih dari 0.6, yang artinya performa model masih mendekati model random guessing. Lalu pada *binary Accuracy* nilai yang dihasilkan juga tidak lebih dari 0.5, serta jumlah *false negative* yang sama *true negatif*, yaitu 23. Sedangkan untuk nilai *false positive* mendapat 11 dan *true positive* mendapat 19, yang artinya model lebih banyak memprediksi nilai yang benar. Terakhir, untuk nilai *binary crossentropy* mendapatkan nilai 1.39626  
| Opsi deployment | Proyek ini di deploy menggunakan Railway App, yaitu salah satu platform yang menyediakan layanan deploy proyek termasuk model machine learning secara gratis |
| Web app |  dapat diakses disini: https://analisis-sentimen-sty-production-ab80.up.railway.app/v1/models/sentimen-analysis-sty-model/metadata |
| Monitoring| Monitoring dilakukan menggunakan Prometheus. Salah satu hal yang dapat dimonitoring adalah *request count* untuk mengetahui jumlah *request* |

# Persiapan Data

In [ ]:
import pandas as pd

df = pd.read_csv('/content/deploy_monitoring_model/data/Analisis_sentimen_timnas_sepakbola_indonesia_di_era_STY.csv')
df.head()

,komentar,label
0,<username> Simulasi indonesia kembali dijajah ...,negatif
1,<username> <username> <username> keturunan lai...,negatif
2,<username> <username> uda botak keturunan penj...,negatif
3,"<username> <username> <username> Belum tentu, ...",negatif
4,<username> <username> <username> Lu nya aja yg...,negatif


In [ ]:
df['label']=[1 if label=='positif' else 0 for label in df['label']]

In [ ]:
import os

folder = '/content/deploy_monitoring_model/final_data/'
os.makedirs(folder, exist_ok=True)
df.to_csv(folder + 'data.csv', index=False, header=True)

# Persiapan

Lingkungan google colab secara otomatis telah menginstall beberapa paket. Maka dari itu kali ini hanya paket yang tidak tersedia saja yang diinstall.

In [ ]:
!pip install tfx

In [ ]:
import os
import sys
from typing import Text

import tensorflow as tf
import tensorflow_model_analysis as tfma
from tfx.components import (
    CsvExampleGen,
    StatisticsGen,
    SchemaGen,
    ExampleValidator,
    Transform,
    Trainer,
    Evaluator,
    Pusher
)
from tfx.proto import example_gen_pb2, trainer_pb2, pusher_pb2
from tfx.types import Channel
from tfx.dsl.components.common.resolver import Resolver
from tfx.types.standard_artifacts import Model, ModelBlessing
from tfx.dsl.input_resolution.strategies.latest_blessed_model_strategy import (
    LatestBlessedModelStrategy)

from absl import logging
from tfx.orchestration import metadata, pipeline
from tfx.orchestration.beam.beam_dag_runner import BeamDagRunner

# Inisialisasi

In [ ]:
PIPELINE_NAME = 'fajar_ramadhan_bbk-pipeline'

DATA_ROOT = '/content/deploy_monitoring_model/final_data'
TRANSFORM_MODULE_FILE = '/content/deploy_monitoring_model/modules/sentimen_analysis_sty_transform.py'
TRAINER_MODULE_FILE = '/content/deploy_monitoring_model/modules/sentimen_analysis_sty_trainer.py'

OUTPUT_BASE = '/content/deploy_monitoring_model/outputs'

SERVING_MODEL_DIR = os.path.join(OUTPUT_BASE, 'serving_model')
PIPELINE_ROOT = os.path.join(OUTPUT_BASE, PIPELINE_NAME)
METADATA_PATH = os.path.join(PIPELINE_ROOT, 'metadata.sqlite')

In [ ]:
"""Initiate tfx pipeline components
"""
def init_components(
    data_dir,
    transform_module,
    training_module,
    training_steps,
    eval_steps,
    serving_model_dir,
):
    """Initiate tfx pipeline components

    Args:
        data_dir (str): a path to the data
        transform_module (str): a path to the transform_module
        training_module (str): a path to the transform_module
        training_steps (int): number of training steps
        eval_steps (int): number of eval steps
        serving_model_dir (str): a path to the serving model directory

    Returns:
        TFX components
    """
    output = example_gen_pb2.Output(
        split_config = example_gen_pb2.SplitConfig(splits=[
            example_gen_pb2.SplitConfig.Split(name="train", hash_buckets=8),
            example_gen_pb2.SplitConfig.Split(name="eval", hash_buckets=2)
        ])
    )

    example_gen = CsvExampleGen(
        input_base=data_dir,
        output_config=output
    )

    statistics_gen = StatisticsGen(
        examples=example_gen.outputs["examples"]
    )

    schema_gen = SchemaGen(
        statistics=statistics_gen.outputs["statistics"]
    )

    example_validator = ExampleValidator(
        statistics=statistics_gen.outputs['statistics'],
        schema=schema_gen.outputs['schema']
    )

    transform = Transform(
        examples=example_gen.outputs['examples'],
        schema=schema_gen.outputs['schema'],
        module_file=os.path.abspath(transform_module)
    )

    trainer  = Trainer(
        module_file=os.path.abspath(training_module),
        examples=transform.outputs['transformed_examples'],
        transform_graph=transform.outputs['transform_graph'],
        schema=schema_gen.outputs['schema'],
        train_args=trainer_pb2.TrainArgs(
            splits=['train'],
            num_steps=training_steps),
        eval_args=trainer_pb2.EvalArgs(
            splits=['eval'],
            num_steps=eval_steps)
    )

    model_resolver = Resolver(
        strategy_class= LatestBlessedModelStrategy,
        model = Channel(type=Model),
        model_blessing = Channel(type=ModelBlessing)
    ).with_id('Latest_blessed_model_resolver')

    slicing_specs=[
        tfma.SlicingSpec(),
        tfma.SlicingSpec(feature_keys=["komentar"])
    ]

    metrics_specs = [
        tfma.MetricsSpec(metrics=[
                tfma.MetricConfig(class_name='ExampleCount'),
                tfma.MetricConfig(class_name='AUC'),
                tfma.MetricConfig(class_name='FalsePositives'),
                tfma.MetricConfig(class_name='TruePositives'),
                tfma.MetricConfig(class_name='FalseNegatives'),
                tfma.MetricConfig(class_name='TrueNegatives'),
                tfma.MetricConfig(class_name='BinaryAccuracy',
                    threshold=tfma.MetricThreshold(
                        value_threshold=tfma.GenericValueThreshold(
                            lower_bound={'value':0.5}),
                        change_threshold=tfma.GenericChangeThreshold(
                            direction=tfma.MetricDirection.HIGHER_IS_BETTER,
                            absolute={'value':0.0001})
                        )
                )
            ])
    ]

    eval_config = tfma.EvalConfig(
        model_specs=[tfma.ModelSpec(label_key='label')],
        slicing_specs=slicing_specs,
        metrics_specs=metrics_specs
    )

    evaluator = Evaluator(
        examples=example_gen.outputs['examples'],
        model=trainer.outputs['model'],
        baseline_model=model_resolver.outputs['model'],
        eval_config=eval_config)

    pusher = Pusher(
        model=trainer.outputs["model"],
        model_blessing=evaluator.outputs["blessing"],
        push_destination=pusher_pb2.PushDestination(
            filesystem=pusher_pb2.PushDestination.Filesystem(
                base_directory=serving_model_dir
            )
        ),
    )

    components = (
        example_gen,
        statistics_gen,
        schema_gen,
        example_validator,
        transform,
        trainer,
        model_resolver,
        evaluator,
        pusher
    )

    return components

In [ ]:
def init_local_pipeline(
    components, pipeline_root: Text
) -> pipeline.Pipeline:

    logging.info(f"Pipeline root set to: {PIPELINE_ROOT}")
    beam_args = [
        "--direct_running_mode=multi_processing"
        # 0 auto-detect based on on the number of CPUs available
        # during execution time.
        "----direct_num_workers=0"
    ]

    return pipeline.Pipeline(
        pipeline_name=PIPELINE_NAME,
        pipeline_root=PIPELINE_ROOT,
        components=components,
        enable_cache=True,
        metadata_connection_config=metadata.sqlite_metadata_connection_config(
            METADATA_PATH
        ),
        beam_pipeline_args=beam_args
    )

if __name__ == "__main__":
    logging.set_verbosity(logging.INFO)

    components = init_components(
        DATA_ROOT,
        training_module=TRAINER_MODULE_FILE,
        transform_module=TRANSFORM_MODULE_FILE,
        training_steps=8,
        eval_steps=8,
        serving_model_dir=SERVING_MODEL_DIR,
    )

    pipeline = init_local_pipeline(components, PIPELINE_ROOT)
    BeamDagRunner().run(pipeline=pipeline)

INFO:absl:Excluding no splits because exclude_splits is not set.
INFO:absl:Excluding no splits because exclude_splits is not set.
INFO:absl:Excluding no splits because exclude_splits is not set.
INFO:absl:Pipeline root set to: /content/deploy_monitoring_model/outputs/fajar_ramadhan_bbk-pipeline
INFO:absl:Generating ephemeral wheel package for '/content/deploy_monitoring_model/modules/sentimen_analysis_sty_transform.py' (including modules: ['sentimen_analysis_sty_trainer', 'sentimen_analysis_sty_transform']).
INFO:absl:User module package has hash fingerprint version a17ab5ee1af37575d77c256c4d34adc0bd60f21f6066fbdc46c775ffe8d84f12.
INFO:absl:Executing: ['/usr/bin/python3', '/tmp/tmprttvt20t/_tfx_generated_setup.py', 'bdist_wheel', '--bdist-dir', '/tmp/tmp_9h_9u0y', '--dist-dir', '/tmp/tmp0ui_kvra']
INFO:absl:Successfully built user code wheel distribution at '/content/deploy_monitoring_model/outputs/fajar_ramadhan_bbk-pipeline/_wheels/tfx_user_code_Transform-0.0+a17ab5ee1af37575d77c256c

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 komentar_xf (InputLayer)    [(None, 1)]               0         
                                                                 
 tf.reshape (TFOpLambda)     (None,)                   0         
                                                                 
 text_vectorization (TextVe  (None, 20)                0         
 ctorization)                                                    
                                                                 
 embedding (Embedding)       (None, 20, 32)            32000     
                                                                 
 global_average_pooling1d (  (None, 32)                0         
 GlobalAveragePooling1D)                                         
                                                                 
 dense (Dense)               (None, 64)                2112  

INFO:absl:Function `_wrapped_model` contains input name(s) table_handle, 2882, resource with unsupported characters which will be renamed to model_text_vectorization_string_lookup_none_lookup_lookuptablefindv2_table_handle, model_embedding_embedding_lookup_2882, model_dense_2_biasadd_readvariableop_resource in the SavedModel.
INFO:absl:Found untraced functions such as _update_step_xla while saving (showing 1 of 1). These functions will not be directly callable after loading.
INFO:absl:Writing fingerprint to /content/deploy_monitoring_model/outputs/fajar_ramadhan_bbk-pipeline/Trainer/model/13/Format-Serving/fingerprint.pb


8/8 [==============================] - 4s 350ms/step - loss: 0.6785 - binary_accuracy: 0.6055 - val_loss: 0.7035 - val_binary_accuracy: 0.5234
Epoch 2/10
1/8 [==>...........................] - ETA: 0s - loss: 0.6510 - binary_accuracy: 0.7188
Epoch 2: val_binary_accuracy did not improve from 0.52344
8/8 [==============================] - 0s 17ms/step - loss: 0.5615 - binary_accuracy: 0.7598 - val_loss: 0.8438 - val_binary_accuracy: 0.5137
Epoch 3/10
7/8 [=========================>....] - ETA: 0s - loss: 0.4020 - binary_accuracy: 0.8504
Epoch 3: val_binary_accuracy did not improve from 0.52344
8/8 [==============================] - 0s 17ms/step - loss: 0.3904 - binary_accuracy: 0.8496 - val_loss: 1.0476 - val_binary_accuracy: 0.4961
Epoch 4/10
1/8 [==>...........................] - ETA: 0s - loss: 0.3335 - binary_accuracy: 0.8750
Epoch 4: val_binary_accuracy improved from 0.52344 to 0.53320, saving model to /content/deploy_monitoring_model/outputs/fajar_ramadhan_bbk-pipeline/Trainer/mode

INFO:absl:Function `_wrapped_model` contains input name(s) table_handle, 4946, resource with unsupported characters which will be renamed to model_text_vectorization_string_lookup_hash_table_lookup_lookuptablefindv2_table_handle, model_embedding_embedding_lookup_4946, model_dense_2_biasadd_readvariableop_resource in the SavedModel.
INFO:absl:Found untraced functions such as _update_step_xla while saving (showing 1 of 1). These functions will not be directly callable after loading.
INFO:absl:Writing fingerprint to /content/deploy_monitoring_model/outputs/fajar_ramadhan_bbk-pipeline/Trainer/model/13/Format-Serving/fingerprint.pb


8/8 [==============================] - 1s 168ms/step - loss: 0.2610 - binary_accuracy: 0.8984 - val_loss: 1.3690 - val_binary_accuracy: 0.5332
Epoch 5/10
1/8 [==>...........................] - ETA: 0s - loss: 0.3036 - binary_accuracy: 0.8594
Epoch 5: val_binary_accuracy did not improve from 0.53320
8/8 [==============================] - 0s 17ms/step - loss: 0.1757 - binary_accuracy: 0.9395 - val_loss: 1.7666 - val_binary_accuracy: 0.5039
Epoch 6/10
1/8 [==>...........................] - ETA: 0s - loss: 0.2092 - binary_accuracy: 0.9062
Epoch 6: val_binary_accuracy did not improve from 0.53320
8/8 [==============================] - 0s 17ms/step - loss: 0.1067 - binary_accuracy: 0.9570 - val_loss: 2.0320 - val_binary_accuracy: 0.5195
Epoch 7/10
1/8 [==>...........................] - ETA: 0s - loss: 0.0504 - binary_accuracy: 1.0000


Epoch 7: val_binary_accuracy improved from 0.53320 to 0.56055, saving model to /content/deploy_monitoring_model/outputs/fajar_ramadhan_bbk-pipeline/Trainer/model/13/Format-Serving


INFO:absl:Function `_wrapped_model` contains input name(s) table_handle, 6795, resource with unsupported characters which will be renamed to model_text_vectorization_string_lookup_hash_table_lookup_lookuptablefindv2_table_handle, model_embedding_embedding_lookup_6795, model_dense_2_biasadd_readvariableop_resource in the SavedModel.
INFO:absl:Found untraced functions such as _update_step_xla while saving (showing 1 of 1). These functions will not be directly callable after loading.
INFO:absl:Writing fingerprint to /content/deploy_monitoring_model/outputs/fajar_ramadhan_bbk-pipeline/Trainer/model/13/Format-Serving/fingerprint.pb


8/8 [==============================] - 2s 229ms/step - loss: 0.0561 - binary_accuracy: 0.9922 - val_loss: 2.7973 - val_binary_accuracy: 0.5605


INFO:absl:Feature komentar has a shape dim {
  size: 1
}
. Setting to DenseTensor.
INFO:absl:Feature label has a shape dim {
  size: 1
}
. Setting to DenseTensor.
INFO:absl:Function `serve_tf_examples_fn` contains input name(s) table_handle, 7828, resource with unsupported characters which will be renamed to model_text_vectorization_string_lookup_hash_table_lookup_lookuptablefindv2_table_handle, model_embedding_embedding_lookup_7828, model_dense_2_biasadd_readvariableop_resource in the SavedModel.
INFO:absl:Found untraced functions such as _update_step_xla while saving (showing 1 of 1). These functions will not be directly callable after loading.
INFO:absl:Writing fingerprint to /content/deploy_monitoring_model/outputs/fajar_ramadhan_bbk-pipeline/Trainer/model/13/Format-Serving/fingerprint.pb
INFO:absl:Training complete. Model written to /content/deploy_monitoring_model/outputs/fajar_ramadhan_bbk-pipeline/Trainer/model/13/Format-Serving. ModelRun written to /content/deploy_monitoring_m